# Ingesting realtime tweets using Apache Kafka

Purpose:
    1)Main data source for the lambda pipline
    2)Uses twitter streaming API to simulate new events coming in every minute
    3)Kafka Producer sends the tweets as records to the kafka Broker
    
Contents:
    1)Twitter setup
    2)Defining the Kafka producer
    3)Producing and sending records to the Kafka Broker
    4)Deployment 

In [2]:
import tweepy
import time
from kafka import KafkaConsumer, KafkaProducer

Twitter setup:
    
    Getting the API object using authorization information

In [3]:
# Twitter developer Credentials to connect to twitter account
consumer_key="jdlA79ZiG76OmdWYKSPRFCeHy"
consumer_secret="JD145Mgq1ymAUuUdbFeGNpuwzslPDfVMiDntog71lelGKBT2DP"
access_token="1314937881048170496-8KvNVPLVJa4c9X3R35UaYLXmUtGvIr"
access_secret="XPxyYNeCG5dCRaSnwX1LFoqsHsUZk1CyL4DLxrZeWFNYH"

In [4]:
#disastertweetsdemo
#Creating the authentication object
auth=tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_secret)
api=tweepy.API(auth)

In [20]:
#from datetime import 
from datetime import datetime, timedelta
def normalize_timestamp(time):
    mytime=datetime.strptime(time,"%Y-%m-%d %H:%M:%S")
    mytime+= timedelta(hours=1)   #the tweets are timestamped in GMT timezone, while I am in +1 timezone
    return (mytime.strftime("%Y-%m-%d %H:%M:%S"))

Defining the Kafka producer

Specify the Kafka Broker

Specify the topic name

Optional: Specify partioning strategy

In [5]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')
topic_name = 'tweets-lambda1'

# Producing and sending records to the Kafka Broker

Resulting tweets have following attributes:

    id , created_at ,followers_count ,location , favourite_count ,retweet_count

In [26]:
def get_twitter_data():
    res=api.search("Earthquake OR Wildfire OR Hurricane")
    for i in res:
        record=''
        record+=str(i.user.id_str)
        record+=';'
        record+=str(normalize_timestamp(str(i.created_at)))
        record+=';'
        record+=str(i.user.followers_count)
        record+=";"
        record+=str(i.user.location)
        record+=";"
        record+=str(i.favorite_count)
        record+=";"
        record+=str(i.retweet_count)
        record+=";"
        producer.send(topic_name, str.encode(record))        
        

In [27]:
get_twitter_data()

# DEPLOYMENT

In [28]:
def periodic_work(interval):
    while True:
        get_twittwer_data()
        #interval should be an integer, the number of seconds to wait
        time.sleep(interval)
        

In [ ]:
periodic_work(60*0.1) #get data every couple of minutes